# Adaptive rejection sampling

Rejection sampling (RS) is a useful method for sampling intractable distributions. It defines an envelope function which upper-bounds the target unnormalised probability density to be sampled. It then proceeds to sample points in the area under the envelope, rejecting those points which fall above the target and accepting the rest. The accepted points are independent and identically distributed samples from the target distribution. There are two important issues with RS. The first is that if the envelope is a very loose upper bound, then most samples will be rejected and the scheme will be slow. The second is that for rejection sampling to work, we must be certain that the envelope is an upper bound to the target, which in practice may be a challenging task.

Adaptive rejection sampling (ARS) {cite}`gilks1992ars` is an efficient method for sampling log-concave targets, which deals with both of these issues. It is origially defined for univariate distributions, but can also be extended to multivariate distributions via Gibbs sampling.{cite}`bishop2006PRML` ARS maintains an envelope which adapts as more points are sampled, becoming a progressively tighter bound to the target, thereby avoiding the inefficiency of regular RS. Further, the way that ARS constructs this envelope guarantees that the envelope is in fact an upper bound to the target, which sidesteps the second difficulty described above.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import HTML, set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
css_style = open('../../../_static/custom_style.css', 'r').read()
HTML(f'<style>{css_style}</style>')

## An adaptive envelope function

Suppose we wish to sample from a log-convex univariate distribution with unnormalised distribution function $f$. Whereas RS defines a fixed envelope, ARS will define an envelope $g_u$ that upper bounds $h = \log f$ and adapts to its shape as the sampling procedure progresses. In addition to this, ARS can use an optional function $g_l$ which lower bounds $\log f$, called the squeezing function. The squeezing function can be used to avoid evaluating $f$ in the rejection step, which can be especially useful if $f$ is computationally expensive to evaluate.

Given the an ordered set of points $x_1 < x_2 < ... < x_K$, ARS defines the envelope $g_u$ to be the minimum over the tangents to $h$ at these points. The squeezing function is defined to be the piecewise linear function which joins the points $(x_k, h(x_k))$ inside the interval $[x_1, x_K]$ and is equal to $-\infty$ outside this innterval. Examples of envelope and squeezing functions are shown below.

<div class="definition">
    
**Definition (Abscissa set, envelope function and squeezing function)** Let $f(x)$ be a univariate log-concave function, with non-zero domain $D = \{x : f(x) > 0\}$. An abscissa set $T_K$ is an ordered set of points in $D$ such that
    
$$T_k = \{x_1 < x_2 < ... < x_K\}.$$
    
The envelope function $g_u(x)$ defined by $T_k$ is
    
$$g_u(x) = \min_{k} g_{u, k}(x)$$
    
where $g_{u, k}(x), k = 1, 2, ..., K$ are piecewise linear functions such that
    
$$g_{u, k}(x_k) = \log f(x_k) \text{ and } g_{u, k}'(x_k) = \log f'(x_k).$$
    
The squeezing function $g_l(x)$ defined by $T_k$ is
    
$$g_l(x) = \begin{cases} \min_{k} g_{l, k}(x)  & \text{ if } x_1 \leq x \leq x_k, \\ -\infty & \text{ otherwise.} \end{cases}$$
    
where $g_{u, k}(x), k = 1, 2, ..., K - 1$ are piecewise linear functions such that
    
$$g_{l, k}(x_k) = \log f(x_k) \text{ and } g_{l, k}(x_{k+1}) = \log f(x_{k+1}).$$
    
</div>
<br>

<div class="definition">
    
**Algorithm (Adaptive Rejection Sampling)** Given a univariate un-normalised probability density $f(x)$, perform the following initialisation, sampling and update steps:
    
1. Initialise an abscissa set $T_k$, such that $f'(x_1) > 0$ and $f'(x_k) < 0$, as well as the corresponding envelope and squeezing functions $g_u$ and $g_l$. This can be efficiently achieved by starting from an initial guess and stepping out in steps of exponentially increasing size.
2. Sample \\[x' \sim \frac{\exp(g_u(x))}{\int \exp(g_u(x')) dx'} \text{ and } z \sim \text{Unifrom}(0, 1),\\] and perform the following squeezing and rejection tests. If \\[z \leq \exp(g_l(x') - g_u(x'))\\] holds, then accept $x'$ otherwise perform the following rejection test \\[z \leq \exp(h(x') - g_u(x'))\\] If this holds, accept the point and otherwise reject it.
3. If $x'$ was accepted at the squeezing test, go to step 2 immediately. Otherwise insert $x'$ into $T_k$ to obtain $T_{k+1}$, update the piecewise exponential functions $g_l$ and $g_u$ accordingly and then return to step 2.
    
</div>
<br>



In [ ]:
def g_u(x, xs, hs, dhdxs):
    pass

def g_l(x, xs, hs):
    pass

def sample_envelope(x, h, dhdx):
    
    # y-intercepts of linear segments of the envelope function
    c = h - dhdx * x
    
    # Points where the linear segments of the envelope function intersect
    z = (c[1:] - c[:-1]) / (dhdx[:-1] - dhdx[1:])
    
    # Integration limits for each piece in the piecewise linear approximation
    limits = np.concatenate([[float('-inf')], z, [float('inf')]])
    limits = np.stack([[limits[:-1], [limits[1:]]]], axis=-1)
    
    probs = 1 / dhdx * (np.exp(limits[:, 1]) - np.exp(limits[:, 0])) * np.exp(c)
    
    # Randomly chosen interval in which the sample lies
    i = np.random.choice(np.arange(probs.shape[0]), p=probs)
    
    # Separately handle the case i = 0
    if i == 0:
        pass
    else:
        pass

## References

```{bibliography} ./ars.bib
```